In [1]:
import pandas as pd
import numpy as np
import io
import sys
import os.path
import urllib.request
from tqdm import tqdm
from os import listdir
from PIL import Image
import glob

pd.set_option('display.max_colwidth', -1)
np.set_printoptions(threshold=sys.maxsize)

In [2]:
df = pd.read_csv("MovieGenre_cleaned.csv",encoding='ISO-8859-1')
df.head()

,imdbId,Title,Genre
0,114709,Toy Story (1995),Animation|Adventure|Comedy
1,113497,Jumanji (1995),Action|Adventure|Family
2,113228,Grumpier Old Men (1995),Comedy|Romance
3,114885,Waiting to Exhale (1995),Comedy|Drama|Romance
4,113041,Father of the Bride Part II (1995),Comedy|Family|Romance


In [3]:
df.shape

(37495, 3)

In [4]:
# How many images actually downloaded

image_list = []
for file in glob.glob("Posters/*.jpg"):
    image_list.append(file)
    
print("Number of files found:", len(image_list))

Number of files found: 36918


In [ ]:
# Resolve mis-match
# Create new dataframe with paths of found images and fill with corresponding metadata from dataframe. 

image_paths = []
imdb_id = []
genres = []
titles = []

for file in glob.glob("Posters/*.jpg"):
    
    #img_id = file.split('.')[0]    
    img_id = file[file.find('/')+1 : file.find('.')]
    title = df[df["imdbId"] == int(img_id)]["Title"].values[0]
    genre = df[df["imdbId"] == int(img_id)]["Genre"].values[0]
    
    image_paths.append(file)
    imdb_id.append(img_id)
    genres.append(genre)
    titles.append(title)
    
    
df_2 = pd.DataFrame({'imdbId': imdb_id, 'Genre': genres, 'Title': titles, 'Image_Paths': image_paths})


In [19]:
print(df_2.shape)
df_2.head()

(36918, 4)


,Image_Paths,imdbId,Genre,Title
0,Posters/27428.jpg,27428,Crime|Mystery|Drama,The Case of the Black Cat (1936)
1,Posters/972555.jpg,972555,Crime|Drama|Thriller,Exhibit A (2007)
2,Posters/1424062.jpg,1424062,Documentary,The Thorn in the Heart (2009)
3,Posters/1185371.jpg,1185371,Documentary,Corman's World: Exploits of a Hollywood Rebel (2011)
4,Posters/113369.jpg,113369,Drama,I Shot a Man in Vegas (1995)


In [12]:
columnsTitles = ['Image_Paths','imdbId','Genre','Title']
df_2 = df_2.reindex(columns=columnsTitles)
df_2.head()

,Image_Paths,imdbId,Genre,Title
0,Posters/27428.jpg,27428,Crime|Mystery|Drama,The Case of the Black Cat (1936)
1,Posters/972555.jpg,972555,Crime|Drama|Thriller,Exhibit A (2007)
2,Posters/1424062.jpg,1424062,Documentary,The Thorn in the Heart (2009)
3,Posters/1185371.jpg,1185371,Documentary,Corman's World: Exploits of a Hollywood Rebel (2011)
4,Posters/113369.jpg,113369,Drama,I Shot a Man in Vegas (1995)


In [13]:
df_2.to_csv("MovieGenre_final.csv", index = None)

In [47]:
# Investigating which of the Imdb_Id of the original csv, did not have a corresponding image

Original_ID_list = np.asarray(df['imdbId']).astype(int)
ID_with_images_found = np.asarray(df_2['imdbId']).astype(int)

#Original lengths of the two arrays 
print("ORIGINAL:", len(Original_ID_list))
print("TRIMMED:", len(ID_with_images_found))

#First compare the unique values of these two arrays
uniq1, counts1 = np.unique(Original_ID_list, return_counts=True)
print("#unique ID values in ORIGINAL:", len(uniq1))

uniq2, counts2 = np.unique(ID_with_images_found, return_counts=True)
print("#unique ID values in TRIMMED:", len(uniq2))

ORIGINAL: 37495
TRIMMED: 36918
#unique ID values in ORIGINAL: 36918
#unique ID values in TRIMMED: 36918


In [51]:
#Find the ID values which are non-unique
non_unique_id = []
for i in range(len(counts1)):
    
    if (counts1[i]>1):
        #print(uniq1[i], "with freq=", counts1[i])
        non_unique_id.append(uniq1[i])
        
#print(non_unique_id)

#Find rows where the column value of imdb is among these non_unique_id values
for i in range(len(non_unique_id)):
    
    print(df.loc[df['imdbId'] == int(non_unique_id[i])])

       imdbId             Title                 Genre
11464  11358   Just Pals (1920)  Comedy|Drama|Western
12042  11358   Just Pals (1920)  Comedy|Drama|Western
       imdbId              Title                     Genre
11386  13556   Robin Hood (1922)  Adventure|Family|Romance
11964  13556   Robin Hood (1922)  Adventure|Family|Romance
       imdbId             Title            Genre
11465  17463   3 Bad Men (1926)  Romance|Western
12043  17463   3 Bad Men (1926)  Romance|Western
       imdbId            Title                    Genre
11608  18192   Napoleon (1927)  Biography|Drama|History
12186  18192   Napoleon (1927)  Biography|Drama|History
       imdbId             Title      Genre
11528  18909   Four Sons (1928)  Drama|War
12106  18909   Four Sons (1928)  Drama|War
       imdbId                   Title                   Genre
11530  18972   Hangman's House (1928)  Drama|Romance|Thriller
12108  18972   Hangman's House (1928)  Drama|Romance|Thriller
       imdbId                  

In [ ]:
#Therefore, we conclude that the duplicate IMDB IDs are now removed and we have only unique IDs with their 
#corresponding posters 
#Next : Organise_data with "MovieGenre_final.csv"